In [1]:
# -*- coding: utf-8 -*- 
# coding:unicode_escape

In [2]:
%%capture cap 
!conda env list
#列出所有 env, 為了找出目前的 env 位置

In [3]:
cap.stdout.split()

['#',
 'conda',
 'environments:',
 '#',
 'base',
 'C:\\ProgramData\\Anaconda3',
 'climate_Taiwan',
 'D:\\Users\\ntuhuser.NTUH\\.conda\\envs\\climate_Taiwan',
 'medcalcR',
 '*',
 'D:\\Users\\ntuhuser.NTUH\\.conda\\envs\\medcalcR',
 'ntuh_captcha',
 'D:\\Users\\ntuhuser.NTUH\\.conda\\envs\\ntuh_captcha']

In [4]:
env_directory_index = cap.stdout.split().index('*')+1 #星號的下一個就是目前的 env 位置
env_directory = cap.stdout.split()[env_directory_index]
env_directory

'D:\\Users\\ntuhuser.NTUH\\.conda\\envs\\medcalcR'

In [5]:
#前置步驟
#conda install r
#conda install pandas
#D:\Users\l1204366\anaconda3\envs\climate_Taiwan/Scripts/pip install rpy2 #根據env位置而定

import os
os.environ["R_HOME"] = env_directory + r'\Lib\R'
os.environ["PATH"]   = env_directory + r'\Lib\R\bin\x64' + ";" + os.environ["PATH"]

In [6]:
%load_ext rpy2.ipython

D:\Users\ntuhuser.NTUH\.conda\envs\medcalcR\lib\site-packages\rpy2\robjects\packages.py:365: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [7]:
%%R
R.version.string

[1] "R version 3.6.1 (2019-07-05)"


In [8]:
%%R
## Default repo
local({r <- getOption("repos")
       r["CRAN"] <- "http://cran.csie.ntu.edu.tw/" 
       options(repos=r)
})
## Encoding
options(encoding = "UTF-8")

In [9]:
%%R
# Install and load required R packages
# ipak function: install and load multiple R packages.
# check to see if packages are installed. Install them if they are not, then load them into the R session.
ipak <- function(pkg){
  new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
  if (length(new.pkg)) 
    install.packages(new.pkg, dependencies = TRUE)
  sapply(pkg, require, character.only = TRUE)}

In [10]:
%%R
ipak(c('shinydashboard','shiny','DT','gsheet'))

R[write to console]: Loading required package: shinydashboard

R[write to console]: 
Attaching package: 'shinydashboard'


R[write to console]: The following object is masked from 'package:graphics':

    box


R[write to console]: Loading required package: shiny

R[write to console]: Loading required package: DT

R[write to console]: This version of Shiny is designed to work with 'htmlwidgets' >= 1.5.
    Please upgrade via install.packages('htmlwidgets').

R[write to console]: 
Attaching package: 'DT'


R[write to console]: The following objects are masked from 'package:shiny':

    dataTableOutput, renderDataTable


R[write to console]: Loading required package: gsheet



shinydashboard          shiny             DT         gsheet 
          TRUE           TRUE           TRUE           TRUE 


In [11]:
%%R
ctab = gsheet2tbl("https://docs.google.com/spreadsheets/d/1eU_eywdp52pJzhVHuAEyEc1cmVKE-qj3MHcGkGK33yE/edit#gid=1506178737")
# type:	1~4, corresponding to female_white, female_african, male_white, male_african
# population: female_white, female_african, male_white, male_african
# c_ln_age: coefficient of ln(age)
# c_ln_age_sq: coefficient of ln(age)^2
# c_ln_tcho: coefficient of ln(total cholesterol)
# c_ln_age_ln_tcho: coefficient of ln(age)*ln(total cholesterol)
# c_ln_hdl: coefficient of ln(high-density lipoprotein)
# c_ln_age_ln_hdl: coefficient of ln(age)*ln(high-density lipoprotein)
# c_ln_tsbp: coefficient of ln(treated systolic blood pressure)
# c_ln_age_ln_tsbp: coefficient of ln(age)*ln(treated systolic blood pressure)
# c_ln_sbp: coefficient of ln(systolic blood pressure)
# c_ln_age_ln_sbp: coefficient of ln(age)*ln(systolic blood pressure)
# c_smoker: coefficient of current smoker
# c_ln_age_smoker: coefficient of ln(age) * (current smoker)
# c_dm: coefficient of diabetes
inptab = gsheet2tbl("https://docs.google.com/spreadsheets/d/1eU_eywdp52pJzhVHuAEyEc1cmVKE-qj3MHcGkGK33yE/edit#gid=345204773")
# sex: choose between female and male (biological)
# race: choose between African and other (white, asian, etc.)
# age: input number between 40 and 79
# tcho: input number between 130 and 320	
# hdl: input number between 20 and 100
# sbp: input number between 90 and 200
# htn: choose between 1 (treated for hypertension) and 0 (not treated for hypertension)
# dm: choose between 1 (diabetic) and 0 (not diabetic)
# smoker: choose between 1 (current smoker) and 0 (not current smoker)

In [12]:
%%R
summary(ctab)

      type       population           mean_sum      baseline_survival
 Min.   :1.00   Length:4           Min.   :-29.18   Min.   :0.8954   
 1st Qu.:1.75   Class :character   1st Qu.:  7.36   1st Qu.:0.9096   
 Median :2.50   Mode  :character   Median : 40.36   Median :0.9338   
 Mean   :2.50                      Mean   : 34.54   Mean   :0.9324   
 3rd Qu.:3.25                      3rd Qu.: 67.54   3rd Qu.:0.9566   
 Max.   :4.00                      Max.   : 86.61   Max.   :0.9665   
    c_ln_age        c_ln_age_sq      c_ln_tcho       c_ln_age_ln_tcho
 Min.   :-29.799   Min.   :0.000   Min.   : 0.3020   Min.   :-3.114  
 1st Qu.: -5.598   1st Qu.:0.000   1st Qu.: 0.7805   1st Qu.:-2.776  
 Median :  7.407   Median :0.000   Median : 6.3965   Median :-1.332  
 Mean   :  0.532   Mean   :1.221   Mean   : 6.6588   Mean   :-1.444  
 3rd Qu.: 13.537   3rd Qu.:1.221   3rd Qu.:12.2747   3rd Qu.: 0.000  
 Max.   : 17.114   Max.   :4.884   Max.   :13.5400   Max.   : 0.000  
    c_ln_hdl       c

In [13]:
%%R
summary(inptab)

     sex                race                age          tcho          hdl    
 Length:1           Length:1           Min.   :40   Min.   :210   Min.   :30  
 Class :character   Class :character   1st Qu.:40   1st Qu.:210   1st Qu.:30  
 Mode  :character   Mode  :character   Median :40   Median :210   Median :30  
                                       Mean   :40   Mean   :210   Mean   :30  
                                       3rd Qu.:40   3rd Qu.:210   3rd Qu.:30  
                                       Max.   :40   Max.   :210   Max.   :30  
      sbp           htn          dm        smoker 
 Min.   :150   Min.   :0   Min.   :0   Min.   :1  
 1st Qu.:150   1st Qu.:0   1st Qu.:0   1st Qu.:1  
 Median :150   Median :0   Median :0   Median :1  
 Mean   :150   Mean   :0   Mean   :0   Mean   :1  
 3rd Qu.:150   3rd Qu.:0   3rd Qu.:0   3rd Qu.:1  
 Max.   :150   Max.   :0   Max.   :0   Max.   :1  


In [14]:
%%R
# determine the coefficient by input table variable sex and race
population_type = NULL # population_type = 1~4 corresponding to ctab$type
if (inptab$sex == 'female') {
  population_type <- if(inptab$race == 'other') 1 else 2
  } else {
  population_type <- if(inptab$race == 'other') 3 else 4    
  }
population_type

[1] 1


In [15]:
%%R
# present the coefficients as matrix with dimention 1*13 (r*c)
cmat <- as.matrix(ctab[ctab$type==population_type, ][-1:-4])
cmat

     c_ln_age c_ln_age_sq c_ln_tcho c_ln_age_ln_tcho c_ln_hdl c_ln_age_ln_hdl
[1,]  -29.799       4.884     13.54           -3.114  -13.578           3.149
     c_ln_tsbp c_ln_age_ln_tsbp c_ln_sbp c_ln_age_ln_sbp c_smoker
[1,]     2.019                0    1.957               0    7.574
     c_ln_age_smoker  c_dm
[1,]          -1.665 0.661


In [25]:
%%R

# construct the input matrix with dimention 13*1 (r*c)
inpmat <- matrix(c(1:13), nrow = 13, ncol = 1)
row.names(inpmat) = c("ln_age", "ln_age_sq", "ln_tcho", "ln_age_ln_tcho", "ln_hdl",
                      "ln_age_ln_hdl", "ln_tsbp", "ln_age_ln_tsbp", "ln_sbp", "ln_age_ln_sbp", 
                      "smoker", "ln_age_smoker", "dm") # corresponding to ctab variables with prefix "c_"
inpmat[1,] = log(inptab$age)
inpmat[2,] = log(inptab$age)^2
inpmat[3,] = log(inptab$tcho)
inpmat[4,] = inpmat[1,] * log(inptab$tcho)
inpmat[5,] = log(inptab$hdl)
inpmat[6,] = inpmat[1,] * inpmat[5,]
inpmat[7,] <- if(inptab$htn ==0) 0 else log(inptab$sbp)
inpmat[8,] <- if(inptab$htn ==0) 0 else inpmat[1,] * inpmat[7,]
inpmat[9,] <- if(inptab$htn ==1) 0 else log(inptab$sbp)
inpmat[10,] <- if(inptab$htn ==1) 0 else inpmat[1,] * inpmat[9,]
inpmat[11,] = inptab$smoker
inpmat[12,] = inpmat[1,] * inpmat[11,]
inpmat[13,] = inptab$dm
inpmat

                    [,1]
ln_age          3.688879
ln_age_sq      13.607832
ln_tcho         5.347108
ln_age_ln_tcho 19.724835
ln_hdl          3.401197
ln_age_ln_hdl  12.546607
ln_tsbp         0.000000
ln_age_ln_tsbp  0.000000
ln_sbp          5.010635
ln_age_ln_sbp  18.483630
smoker          1.000000
ln_age_smoker   3.688879
dm              0.000000


In [26]:
%%R
# calculate ASCVD 10-y risk 
sum = cmat %*% inpmat # matrix multiplication of coefficient matrix to input matrix
baseline_survival = ctab$baseline_survival[ctab$type == population_type] # baseline survival according to the population type
mean_sum = ctab$mean_sum[ctab$type == population_type] # mean result of matrix multiplication according to the population type
ascvd_risk = 1-baseline_survival ^ exp(sum - mean_sum)

In [27]:
%%R
# show the result
output_text = 'ASCVD risk: '
Encoding(output_text) <- 'UTF-8'
output_text2 = round(ascvd_risk*100, 2)
output_text3 = ' %'
final_output = paste0(output_text, output_text2, output_text3)
final_output

[1] "ASCVD risk: 11.3 %"


In [24]:
%%R
sum - mean_sum

          [,1]
[1,] -8.547746
